## AC Transit GTFS RT

**Links**: 
- [AC Transit API Documentation](https://api.actransit.org/transit/Help)


- [OpenMobilityData](https://transitfeeds.com/p/ac-transit/579) has their own API but probably should just go to the source via AC Transit
- [Urban Access](https://github.com/UDST/urbanaccess) mentioned in class but not sure if what I am looking for
- how-to on the Python requests library: https://realpython.com/python-requests/


- [Using PostgreSQL in Python](https://www.datacamp.com/community/tutorials/tutorial-postgresql-python)
- Heroku deployment:
    - http://briancaffey.github.io/2016/04/05/twitter-bot-tutorial.html <-- try setting up this tweetbot first and then can change the code to run AC Transit API....would I even need to push the data to PostgreSQL with this method or could it be stored "on file" on Heroku's server?
    - https://bigishdata.com/2016/12/15/running-python-background-jobs-with-heroku/
    - https://www.youtube.com/watch?v=DwWPunpypNA
- [Python Anywhere](https://www.pythonanywhere.com/) could be another option?


**Currently working on**:
- [x] having trouble calling API, maybe can ask Anson (see: [Quantifying Transit Reliability](https://blog.conveyal.com/quantifying-transit-reliability-using-accessibility-indicators-6b69f5337cd3)) --> [SOLVED] [MobilityData](https://github.com/MobilityData/gtfs-realtime-bindings/tree/master/python) has sample code showing how to use the google.transit package to decode the protocol buffer returned by the API
- [x] convert feed.entity to pandas df
- [x] create map (folium?) with locations of current buses
- [ ] schedule realtime calls, collect data
- [ ] identify bus bunching
- [x] create config file with tokens and keys that can be hidden on local machine

In [7]:
import pandas as pd
import numpy as np
import csv
import json
from datetime import datetime
import requests
from google.transit import gtfs_realtime_pb2

In [3]:
from config import *
BASE = "https://api.actransit.org/transit/"

#### Call RT-GTFS

In [4]:
#response = requests.get(config['BASE']+"gtfsrt/vehicles/?token="+config['KEY'])
#response = requests.get(config['BASE']+"actrealtime/servicebulletin/?token="+config['KEY'])

In [5]:
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get(BASE+"gtfsrt/vehicles/?token="+KEY)
feed.ParseFromString(response.content)

20827

In [5]:
#feed.entity

In [6]:
#feed.entity[0].vehicle.trip.route_id
feed.entity[-1]

id: "327"
vehicle {
  trip {
    trip_id: "835834020"
    schedule_relationship: SCHEDULED
    route_id: "62"
  }
  position {
    latitude: 37.77540969848633
    longitude: -122.22554016113281
    bearing: 311.0
    speed: 0.0
  }
  timestamp: 1582060814
  vehicle {
    id: "1449"
  }
}

In [7]:
for entity in feed.entity:
    if entity.vehicle.trip.route_id == "6":
        print(entity.vehicle.trip.trip_id)

743236020
743281020
743300020
743357020
743218020
743322020
743337020
743370020


#### convert feed.entity to pandas df

this and next section on folium built from: https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [24]:
route_id = []
trip_id = []
vehicle_id = []
latitude = []
longitude = []
bearing = []
speed = []
unix_time = []
#schedule_relationship = []

for entity in feed.entity:
    route_id.append(entity.vehicle.trip.route_id)
    trip_id.append(entity.vehicle.trip.trip_id)
    vehicle_id.append(entity.vehicle.vehicle.id)
    latitude.append(entity.vehicle.position.latitude)
    longitude.append(entity.vehicle.position.longitude)
    bearing.append(entity.vehicle.position.bearing)
    speed.append(entity.vehicle.position.speed)
    unix_time.append(entity.vehicle.timestamp)

df_busLocations = pd.DataFrame(
    {'route_id' : route_id,
     'trip_id' : trip_id,
     'vehicle_id' : vehicle_id,
     'latitude' : latitude,
     'longitude' : longitude,
     'bearing' : bearing,
     'speed': speed,
     'unix_time': unix_time
    })

df_busLocations['time'] = pd.to_datetime(df_busLocations['unix_time'], unit='s').astype('datetime64[ns, US/Pacific]')

In [25]:
df_busLocations[df_busLocations.route_id == "80"].head(25)

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed,unix_time,time
0,80,742472020,5020,37.853138,-122.281914,254.0,5.36448,1582060824,2020-02-18 13:20:24-08:00
32,80,742431020,5016,37.868359,-122.297806,166.0,0.00000,1582060817,2020-02-18 13:20:17-08:00
256,80,742471020,5120,37.901131,-122.309242,79.0,9.38784,1582060826,2020-02-18 13:20:26-08:00
262,80,742430020,5004,37.902874,-122.299194,332.0,3.57632,1582060816,2020-02-18 13:20:16-08:00
280,80,742432020,5115,37.856735,-122.255119,79.0,8.04672,1582060823,2020-02-18 13:20:23-08:00


#### use folium to map current bus locations

In [116]:
import folium

In [117]:
locations = df_busLocations[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [118]:
map = folium.Map(location=[37.8, -122.2], tiles='CartoDB positron', zoom_start=11)
for point in range(0, len(locationlist)):
    folium.Marker(
        locationlist[point], 
        popup=folium.Popup(
            "Route: " + df_busLocations['route_id'][point] + "\n" + 
            "Trip: " + df_busLocations['trip_id'][point]
        )
    ).add_to(map)
map

#### save bus locations (lat, lon) to file

In [30]:
df_busLocations.to_csv("ACTransit_Veh_Loc.csv")

In [31]:
df_busLocations.to_csv("ACTransit_Veh_Loc_0219.csv", mode='a', header=False)

#### loop to call API and save everything to file

In [ ]:
def call_api(BASE, KEY):
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get(BASE+"gtfsrt/vehicles/?token="+KEY)
    feed.ParseFromString(response.content)
    return feed

def api_json_to_csv(feed):
    route_id = []
    trip_id = []
    vehicle_id = []
    latitude = []
    longitude = []
    bearing = []
    speed = []
    unix_time = []
    #schedule_relationship = []

    for entity in feed.entity:
        route_id.append(entity.vehicle.trip.route_id)
        trip_id.append(entity.vehicle.trip.trip_id)
        vehicle_id.append(entity.vehicle.vehicle.id)
        latitude.append(entity.vehicle.position.latitude)
        longitude.append(entity.vehicle.position.longitude)
        bearing.append(entity.vehicle.position.bearing)
        speed.append(entity.vehicle.position.speed)
        unix_time.append(entity.vehicle.timestamp)

    df = pd.DataFrame(
        {'route_id' : route_id,
         'trip_id' : trip_id,
         'vehicle_id' : vehicle_id,
         'latitude' : latitude,
         'longitude' : longitude,
         'bearing' : bearing,
         'speed': speed,
         'unix_time': unix_time
        })

# scheduler to call and save API
# while x < 10:
    

#### testing other API Calls

RouteIds

In [18]:
response = requests.get(config['BASE']+"routes/?token="+config['KEY'])
routes = response.json()

In [19]:
route_id = []
name = []
description = []

for i in routes:
    route_id.append(i['RouteId'])
    name.append(i['Name'])
    description.append(i['Description'])

routes_df = pd.DataFrame(
    {'RouteId' : route_id,
     'Name' : name,
     'Description' : description
    })

In [20]:
routes_df

,RouteId,Name,Description
0,1,1,International - E. 14th
1,10,10,E. 14th St. - Mission
2,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.
3,14,14,14th St - San Antonio - High St
4,18,18,Solano - Shattuck - MLK Jr.
5,19,19,Buena Vista - Fruitvale
6,20,20,Dimond - Fruitvale - South Shore
7,200,200,Decoto - Newark Blvd. - Mowry
8,21,21,Dimond - Fruitvale - Bay Farm
9,210,210,Fremont Blvd. - Mission San Jose


route directions

In [21]:
route_names = routes_df['Name']
directions = []
for i in route_names:
    response = requests.get(config['BASE']+"route/"+i+"/directions?token="+config['KEY'])
    dir_response = response.json()
    directions.append(dir_response)

routes_df['Directions'] = directions

In [22]:
routes_df.head()

,RouteId,Name,Description,Directions
0,1,1,International - E. 14th,"[NORTH, Northbound, SOUTH, Southbound]"
1,10,10,E. 14th St. - Mission,"[Northbound, Southbound]"
2,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.,"[NORTH, Northbound, SOUTH, Southbound]"
3,14,14,14th St - San Antonio - High St,"[EAST, Eastbound, WEST, Westbound]"
4,18,18,Solano - Shattuck - MLK Jr.,"[NORTH, Northbound, SOUTH, Southbound]"


In [25]:
unique_dir = []
for i in directions:
    for j in i:
        if j not in unique_dir:
            unique_dir.append(j)
unique_dir

['NORTH',
 'Northbound',
 'SOUTH',
 'Southbound',
 'EAST',
 'Eastbound',
 'WEST',
 'Westbound',
 'Clockwise',
 'CW',
 '1',
 'DIR1',
 'Outbound',
 'CCW',
 'Counterclock']

loop to get route and directions in long format database

In [103]:
# get routes from API
response = requests.get(config['BASE']+"routes/?token="+config['KEY'])
routes = response.json()

# initialize empty arrays for database
route_id = []
name = []
description = []
direction = []

# loop through routes
for i in routes:
    # get directions per route from API
    print("Now processing:{}".format(i['RouteId']))
    response = requests.get(config['BASE']+"route/"+i['RouteId']+"/directions?token="+config['KEY'])
    dir_response = response.json()
    
    # loop through directions
    for j in dir_response:
        route_id.append(i['RouteId'])
        name.append(i['Name'])
        description.append(i['Description'])
        direction.append(j)
    #end for j
#end for i

routes_dir_df = pd.DataFrame(
    {'RouteId': route_id,
     'Name': name,
     'Description': description,
     'Direction': direction
    })

Now processing:1
Now processing:10
Now processing:12
Now processing:14
Now processing:18
Now processing:19
Now processing:20
Now processing:200
Now processing:21
Now processing:210
Now processing:212
Now processing:215
Now processing:216
Now processing:217
Now processing:232
Now processing:239
Now processing:251
Now processing:28
Now processing:29
Now processing:314
Now processing:33
Now processing:339
Now processing:34
Now processing:35
Now processing:356
Now processing:36
Now processing:376
Now processing:39
Now processing:399
Now processing:40
Now processing:41
Now processing:448
Now processing:45
Now processing:46
Now processing:46L
Now processing:47
Now processing:475
Now processing:51A
Now processing:51B
Now processing:52
Now processing:54
Now processing:56
Now processing:57
Now processing:6
Now processing:60
Now processing:604
Now processing:605
Now processing:606
Now processing:607
Now processing:611
Now processing:617
Now processing:62
Now processing:620
Now processing:621
Now

In [104]:
routes_dir_df.head(10)

,RouteId,Name,Description,Direction
0,1,1,International - E. 14th,NORTH
1,1,1,International - E. 14th,Northbound
2,1,1,International - E. 14th,SOUTH
3,1,1,International - E. 14th,Southbound
4,10,10,E. 14th St. - Mission,Northbound
5,10,10,E. 14th St. - Mission,Southbound
6,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.,NORTH
7,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.,Northbound
8,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.,SOUTH
9,12,12,Dtn. Oakland\Dtn. Berkeley\ Gilman St.,Southbound


vehicles
- appears to get real time location of all vehicles on a specific route

In [108]:
#route/{routeName}/vehicles
response = requests.get(config['BASE'] + "route/6/vehicles?token="+config['KEY'])
vehicles = response.json()

In [109]:
map = folium.Map(location=[37.8, -122.2], tiles='CartoDB positron', zoom_start=11)
for veh in vehicles:
    folium.Marker(
        [veh['Latitude'], veh['Longitude']], 
        popup=folium.Popup(
            "Vehicle: " + str(veh['VehicleId']) + "\n" + 
            "Trip: " + str(veh['CurrentTripId']) + "\n" + 
            "Heading: " + str(veh['Heading'])
        )
    ).add_to(map)
map

NameError: name 'folium' is not defined

trips

In [32]:
#route/{routeName}/trips?direction={direction}&scheduleType={scheduleType}
response = requests.get(config['BASE'] + "route/6/trips?direction=Northbound&scheduleType=Weekday?token="+config['KEY'])
data = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
response

<Response [401]>

Realtime Information

In [34]:
df_busLocations[df_busLocations.route_id == "6"]

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed
9,6,743357020,1580,37.850712,-122.260452,187.0,0.00000
36,6,743236020,1218,37.814091,-122.268356,13.0,0.00000
50,6,743218020,1350,37.801842,-122.273636,298.0,0.00000
54,6,743322020,1347,37.867065,-122.258842,349.0,7.15264
64,6,743281020,1343,37.802265,-122.270958,21.0,1.34112
83,6,743300020,1357,37.872200,-122.268295,260.0,0.00000
86,6,743370020,1353,37.845360,-122.261101,358.0,5.81152
268,6,743256020,1553,37.815857,-122.268143,194.0,0.00000
304,6,743337020,1202,37.871342,-122.265816,355.0,0.00000


In [80]:
#response = requests.get(config['BASE'] + "route/6/trip/743281020/stops&token="+config['KEY'])
#response = requests.get(config['BASE'] + "route/6/trip/743281020/stops&token="+config['KEY'])
#response = requests.get(config['BASE'] + "route/6/directions?token="+config['KEY'])
#response = requests.get(config['BASE'] + "route/6?token="+config['KEY'])

In [91]:
#response = requests.get(config['BASE'] + "actrealtime/line?callback=jsonp&token=" + config['KEY'])
response = requests.get(config['BASE'] + "stops?token=" + config['KEY'])

In [93]:
stops = response.json()

In [106]:
#GET actrealtime/stop?rt={rt}&dir={dir}&stpid={stpid}&callback={callback}
routes_dir_6_df = routes_dir_df[routes_dir_df.RouteId == "6"]
for row, idx in routes_dir_6_df.iter():
    rt = row['RouteId']
    dr = row['Direction']
    response = requests.get(config['BASE'] + "actrealtime/stop?rt="+rt+"&dir="+dr+"&token=" + config['KEY'])
    next_stops = response.json()
    print(next_stops)

AttributeError: 'DataFrame' object has no attribute 'iter'

In [101]:
next_stops

{'bustime-response': {'error': [{'rtpidatafeed': 'bustime',
    'rt': '10',
    'dir': 'Northbound',
    'msg': 'No data found for parameter'}]}}

In [94]:
#GET actrealtime/direction?rt={rt}&callback={callback}
response = requests.get(config['BASE'] + "actrealtime/direction?rt=6&token="+config['KEY'])
if response:
    print('Success!') 
    data = response.json()
else:
    print('An error has occurred.')
    print(response.headers)
    print(response.content)

Success!


In [96]:
stops[:5]

[{'StopId': 52246,
  'Name': '8th St:Portola Av',
  'Latitude': 37.7688136,
  'Longitude': -122.2729918,
  'ScheduledTime': None},
 {'StopId': 57793,
  'Name': '8th St:Portola Av',
  'Latitude': 37.7689929,
  'Longitude': -122.2727976,
  'ScheduledTime': None},
 {'StopId': 56707,
  'Name': '1105 Atlantic Av',
  'Latitude': 37.7802724,
  'Longitude': -122.2640149,
  'ScheduledTime': None},
 {'StopId': 57796,
  'Name': 'Atlantic Av:Challenger Dr',
  'Latitude': 37.7810663,
  'Longitude': -122.2700596,
  'ScheduledTime': None},
 {'StopId': 54332,
  'Name': 'Atlantic Av:Triumph Dr',
  'Latitude': 37.780799,
  'Longitude': -122.266125,
  'ScheduledTime': None}]

#### get route-specific patterns

https://api.actransit.org/transit/Help/Api/GET-actrealtime-pattern_pid_rt_callback

In [23]:
# GET actrealtime/pattern?pid={pid}&rt={rt}&callback={callback}
response = requests.get(config['BASE'] + "actrealtime/pattern?rt=6&token="+config['KEY'])
if response:
    print('Success!') 
    patterns = response.json()
else:
    print('An error has occurred.')
    print(response.headers)
    print(response.content)

Success!


In [27]:
patterns['bustime-response']['ptr'][0]

{'pid': 5535,
 'ln': 31559.0,
 'rtdir': 'To Downtown Oakland',
 'pt': [{'seq': 1,
   'lat': 37.871341985008,
   'lon': -122.265813,
   'typ': 'S',
   'stpid': '51705',
   'stpnm': 'Oxford St:Addison St',
   'pdist': 0.0},
  {'seq': 2, 'lat': 37.871653, 'lon': -122.265911, 'typ': 'W', 'pdist': 0.0},
  {'seq': 3, 'lat': 37.872485, 'lon': -122.266001, 'typ': 'W', 'pdist': 0.0},
  {'seq': 4, 'lat': 37.872185, 'lon': -122.268407, 'typ': 'W', 'pdist': 0.0},
  {'seq': 5, 'lat': 37.870808, 'lon': -122.268254, 'typ': 'W', 'pdist': 0.0},
  {'seq': 6, 'lat': 37.870565, 'lon': -122.268187, 'typ': 'W', 'pdist': 0.0},
  {'seq': 7, 'lat': 37.87015, 'lon': -122.267976, 'typ': 'W', 'pdist': 0.0},
  {'seq': 8, 'lat': 37.869762, 'lon': -122.267993, 'typ': 'W', 'pdist': 0.0},
  {'seq': 9, 'lat': 37.869593, 'lon': -122.268073, 'typ': 'W', 'pdist': 0.0},
  {'seq': 10,
   'lat': 37.869479985142,
   'lon': -122.268127000002,
   'typ': 'S',
   'stpid': '55555',
   'stpnm': 'Shattuck Av:Allston Way',
   'pdist'

In [35]:
for i in patterns['bustime-response']['ptr']:
    print("")
    print(i['rtdir'])
    for j in i['pt']:
        try:
            if j['stpid']:
                latlon = (j['lat'], j['lon'])
                print(j['stpnm'], latlon)
        except KeyError:
            continue


To Downtown Oakland
Oxford St:Addison St (37.871341985008, -122.265813)
Shattuck Av:Allston Way (37.869479985142, -122.268127000002)
Shattuck Av + Kittredge St (37.868191985236, -122.267990000002)
Durant Av:Shattuck Av (37.866694985344, -122.267150999999)
Durant Av + Ellsworth St (37.867200985307, -122.263217000002)
Durant Av + Dana St (37.867421985292, -122.261415999999)
Dana St + Haste St (37.865814985408, -122.260992999999)
Telegraph Av + Dwight Way (37.864506985501, -122.258621000001)
Telegraph Av + Parker St (37.862916985618, -122.258881999999)
Telegraph Av + Derby St (37.861156985743, -122.259126)
Telegraph Av + Stuart St (37.859712985847, -122.259331999999)
Telegraph Av + Russell St (37.857788985987, -122.259588999999)
Telegraph Av + Ashby Av (37.856404986087, -122.259786999999)
Telegraph Av + Webster St (37.855314986167, -122.259933999998)
Telegraph Av + Alcatraz Av (37.850385986521, -122.260625)
Telegraph Av + 62nd St (37.848039986691, -122.260955999999)
Telegraph Av + 60th S